In [ ]:
%autosave 60

Autosaving every 60 seconds


In [ ]:
!pip install bitsandbytes
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from transformers.pipelines import pipeline
import bitsandbytes

#### Load the base Model to be finetuned

In [ ]:
model_id="distilbert/distilroberta-base"

In [ ]:
classifier= pipeline('text-classification', model=model_id)

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


Syntax to run model
No meaningful output as of now as this is the base model and there are no lables like "POSITIVE" and "NEGATIVE" that it has inbuilt



In [ ]:
classifier("HELLO!!")

[{'label': 'LABEL_1', 'score': 0.5895799398422241}]

## **CREATING** **DATASET**
#### TODO: Get a much larger and comprehensive dataset

In [ ]:
# load the dataset
# TODO -  generate large synthetic data in future to prevent overfitting
import pandas as pd
df1 = pd.read_csv("/content/jira_scrum_role_dataset.csv")
df2 = pd.read_csv("/content/role_classification_dataset_long.csv")

In [ ]:
#concat the dataset into a single entity with mixing the rows
dataset = pd.concat([df1,df2], ignore_index=True)

In [ ]:
dataset

,description,role
0,Implement responsive UI using React: Implement...,FrontEndEngineer
1,Add new components to the dashboard: Add new c...,FrontEndEngineer
2,Add new components to the dashboard: Add new c...,FrontEndEngineer
3,Fix CSS styling issues for mobile view: Fix CS...,FrontEndEngineer
4,Add new components to the dashboard: Add new c...,FrontEndEngineer
...,...,...
6995,Scale bleeding-edge web-readiness. Develop new...,FrontEndEngineer
6996,Revolutionize scalable solutions. Configure al...,DevOpsEngineer
6997,Syndicate customized paradigms. Improve access...,FrontEndEngineer
6998,Incubate mission-critical architectures. Fix b...,BackendEngineer


In [ ]:
dataset_y = dataset['role']
dataset.drop(['role'], inplace=True, axis = 1)

In [ ]:
dataset_y

,role
0,FrontEndEngineer
1,FrontEndEngineer
2,FrontEndEngineer
3,FrontEndEngineer
4,FrontEndEngineer
...,...
6995,FrontEndEngineer
6996,DevOpsEngineer
6997,FrontEndEngineer
6998,BackendEngineer


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_y, test_size=0.2, random_state=42)

## Preprocessing
- Clean the data (if necessary)
- Need to tokenize the inputs to feed into model
- Need to label encode the output to multiple classes for classification


In [ ]:
cols = y_train.unique()
cols

array(['BackendEngineer', 'FrontEndEngineer', 'CloudEngineer',
       'DevOpsEngineer', 'AIEngineer', 'DatabaseDesignEngineer'],
      dtype=object)

# **TOKENIZING**

##### We tokenize input data using the existing tokenizer for the base model

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

In [ ]:
X_train

,description
1032,Integrate third-party payment gateway: Integra...
6339,Envisioneer front-end e-services. Improve acce...
3886,Optimize cloud resource usage: Optimize cloud ...
2653,Configure Docker containers for services: Conf...
6914,Brand enterprise users. Develop new React comp...
...,...
3772,Migrate services to AWS Lambda: Migrate servic...
5191,Evaluate model performance on validation set: ...
5226,Set up ML pipeline for training and inference:...
5390,Fine-tune GPT model for text generation: Fine-...


In [ ]:
# Tokenising the train and test inputs to finetune the model
train_encodings = tokenizer(list(X_train['description']), padding=True, truncation=True, max_length=256, return_tensors="pt")
test_encodings = tokenizer(list(X_test['description']), padding=True, truncation=True, max_length=256, return_tensors="pt")

#### Since the task is classification, we use a Label Encoder to convert the roles into numeric representations so simplify the process of prediction

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_enc = torch.tensor(label_encoder.fit_transform(y_train))
y_test_enc = torch.tensor(label_encoder.transform(y_test))

In [ ]:
y_train_enc

tensor([1, 5, 2,  ..., 0, 0, 5])

### Create a Role to Index and Role to Index Mapping

In [ ]:
# RoleToIndex = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_).astype(float)))
# IndexToRole = dict(zip(label_encoder.transform(label_encoder.classes_).astype(float), label_encoder.classes_))
IndexToRole = {int(i): role for i, role in zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_)}
RoleToIndex = {role: int(i) for role, i in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}


In [ ]:
RoleToIndex, IndexToRole

({'AIEngineer': 0,
  'BackendEngineer': 1,
  'CloudEngineer': 2,
  'DatabaseDesignEngineer': 3,
  'DevOpsEngineer': 4,
  'FrontEndEngineer': 5},
 {0: 'AIEngineer',
  1: 'BackendEngineer',
  2: 'CloudEngineer',
  3: 'DatabaseDesignEngineer',
  4: 'DevOpsEngineer',
  5: 'FrontEndEngineer'})

The Roberta Base Model does not come with any predefined labels like "POSITIVE" "NEGATIVE". This is becase it has not been trained for any specific purpose.

We can define our own labels and can finetune the model on those classes.

The following cell updates the base model with custom labels define above

In [ ]:
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": IndexToRole})

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Converting the dataset to the standard format required for the training and finetuning of Roberta Models using pytorch

In [ ]:
from torch.utils.data import Dataset,DataLoader
class RoleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RoleDataset(train_encodings, y_train_enc)
test_dataset = RoleDataset(test_encodings, y_test_enc)

## **FINETUNING** **BEGINS**

We will save multiple versions of the model in a directory to and pick the model with the least loss

In [ ]:
import time
output_dir = f'./summary-training'

In [ ]:
training_args = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs=2, #entire dataset will be trained on twice
    # warmup_steps=1, # gradually increases learning rate from 0 to alpha in <warmp_steps> steps. Currently not useful
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    # max_steps=1000,  # can override num.epochs
    learning_rate=1e-5,
    optim="paged_adamw_8bit", #optimiser used for gradient descent
    logging_strategy="steps",
    logging_steps=100, # prints the loss every 100 steps
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=500, # Saves the model every 500 steps
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
100,1.397500
200,0.249600
300,0.077200
400,0.034000
500,0.007200
600,0.016200
700,0.003000
800,0.002200
900,0.001800
1000,0.001600


TrainOutput(global_step=2800, training_loss=0.0644234616934721, metrics={'train_runtime': 513.2685, 'train_samples_per_second': 21.821, 'train_steps_per_second': 5.455, 'total_flos': 214446897907200.0, 'train_loss': 0.0644234616934721, 'epoch': 2.0})

## ***NOTE***

- Loss is much less than it should be because of Model Over-Fitting.

- This can be avoided by a larger dataset with more diverse examples

### After training, load the model from directroy

In [ ]:
model_path = "/content/summary-training/checkpoint-2800"
model = RobertaForSequenceClassification.from_pretrained(model_path)

We require the tokenizer used to tokenize the input text and feed into the model

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

#### Testing on a random entry from test dataset

---



In [ ]:
input = list(X_test['description'])
X_test

,description
6500,Re-intermediate e-business bandwidth. Fix bugs...
2944,Monitor server metrics with Prometheus: Monito...
2024,Monitor server metrics with Prometheus: Monito...
263,Fix CSS styling issues for mobile view: Fix CS...
4350,Normalize database schema: Normalize database ...
...,...
3484,Set up cloudwatch alarms for EC2 instances: Se...
1860,Implement RESTful APIs with Node.js: Implement...
4974,Add indexes to frequently queried columns: Add...
387,Refactor frontend state management: Refactor f...


In [ ]:
text = input[4]

In [ ]:
inputs = tokenizer(text, padding=True, truncation=True, max_length=256, return_tensors="pt")

In [ ]:
# Tokenized input string
inputs

{'input_ids': tensor([[    0, 45647,  2072,  8503, 47404,    35, 26411,  2072,  8503, 47404,
             4, 28688,  2072, 41614, 22680,    13,   819,     4, 38141,   275,
          3464,    32,  1432,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [ ]:
# Code to get models output
import torch

model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()


In [ ]:
predicted_role = IndexToRole[predicted_class_id]
print("Predicted role:", predicted_role)

Predicted role: DatabaseDesignEngineer


In [ ]:
def predict(text, model, tokenizer):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=256, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()
        predicted_role = IndexToRole[predicted_class_id]
        # print("Predicted Role", predicted_role)
        return predicted_role

Testing the Model:

In [ ]:
def test_metric(model, X_test, y_test, tokenizer):
    from sklearn.metrics import accuracy_score
    import torch
    predictions = []
    model.eval()  # Set model to evaluation mode

    for text in X_test['description']:
        predicted_role = predict(text, model, tokenizer)
        predictions.append(predicted_role)

    return [predictions, accuracy_score(y_test, predictions)]

In [ ]:
# test_metric(model, X_test, y_test)

predictions, score = test_metric(model, X_test, y_test, tokenizer)
# print(predictions)
# print("accuracy : ", score)
# print(X_test)
# X_test
# y_test

,description
6500,Re-intermediate e-business bandwidth. Fix bugs...
2944,Monitor server metrics with Prometheus: Monito...
2024,Monitor server metrics with Prometheus: Monito...
263,Fix CSS styling issues for mobile view: Fix CS...
4350,Normalize database schema: Normalize database ...
...,...
3484,Set up cloudwatch alarms for EC2 instances: Se...
1860,Implement RESTful APIs with Node.js: Implement...
4974,Add indexes to frequently queried columns: Add...
387,Refactor frontend state management: Refactor f...


(Again, overfitting)

In [ ]:
score

1.0

In [ ]:
text = """
    Design a database schema to store user and admin data
"""
predict(text, model, tokenizer)

'DatabaseDesignEngineer'

In [ ]:
text = """
    Title:
Refactor Product Detail Page to Use Modular React Components with Tailwind Support

Description:
The current product detail page is a mix of outdated class-based React components and inline CSS, leading to poor maintainability and inconsistent design. Refactor the page using modern React functional components and hooks. Break the UI into reusable components like ProductImageCarousel, PriceTag, AddToCartButton, and ReviewSection.

Implement responsive styling using Tailwind CSS and ensure accessibility compliance (WCAG AA). Also, add client-side validation for the quantity selector and implement skeleton loaders for API-bound data (product details and reviews).

Collaborate with the backend/devops team to adjust the API payload shape for better consumption and integrate the final design with the current Redux store or context API setup. This update is expected to reduce page load times and improve maintainability.

"""
predict(text, model, tokenizer)

'FrontEndEngineer'

In [ ]:
text = """
Title:
Implement Audit Log Viewer with Pagination and Export Support

Description:
We need to add a new page to the admin dashboard where users with proper privileges can view system-wide audit logs. The frontend should provide filters (by user, action, and date), infinite scroll/pagination, and an export-to-CSV option.

The backend endpoint for fetching logs is ready but currently returns raw JSON. You may need to adjust the format slightly and ensure it's paginated efficiently. On the frontend, use either React Table or a custom solution to display entries, including timestamp, username, IP address, and action type.

Ensure the page is styled consistently with the rest of the admin dashboard (Tailwind-based), and hook the export button to a download function using Blob. Add basic loading indicators and error boundaries. Work with DevOps if needed to enable API access via the internal gateway.
"""
predict(text, model, tokenizer)

'DatabaseDesignEngineer'

In [ ]:
text = """
Design and Implement Rate Limiting Middleware for Public API Endpoints

Description:
Several of our public-facing REST API endpoints are currently unauthenticated and are vulnerable to abuse from high-frequency clients. Implement a robust rate-limiting solution using a token bucket algorithm or a sliding window log strategy, configurable per endpoint.

The middleware should be written in Node.js (Express) and support Redis as a shared rate-limit state store across instances. Integrate the middleware into the existing API gateway, and ensure it is configurable via environment variables.

Additionally, expose internal metrics (current token count, IP request count, reset time) via a /metrics route for Prometheus scraping. Add unit tests to cover edge cases like key expiration, sudden burst loads, and distributed sync issues.

Collaborate with the DevOpsEngineer to provision Redis in the Kubernetes cluster and with the FrontEndEngineer to define how rate-limit errors should be surfaced in the UI (e.g., 429 with retry-after header).
"""
predict(text, model, tokenizer)

'BackendEngineer'

In [ ]:
text = """
Title:
Develop Product Recommendation Engine Using User Interaction Embeddings

Description:
We want to build a personalized product recommendation engine that can suggest items based on user interaction history across the site (clicks, views, cart adds). The solution should involve embedding user-item interactions using a suitable architecture (e.g., collaborative filtering with embeddings, transformer-based recommender, or matrix factorization).

Train the model on historical logs (approx. 50M interactions) and fine-tune it to optimize for precision@10. You may use PyTorch or TensorFlow depending on familiarity. Once trained, the model should be served via a REST endpoint with a latency budget of 100ms.

Coordinate with FrontEndEngineers to test model integration and API response formatting, and with DevOps for deployment on the model-serving cluster (TorchServe or TensorFlow Serving preferred).
"""
predict(text, model, tokenizer)

'AIEngineer'

In [ ]:
text = """
Title:
Refactor Sharded Multi-Tenant Reporting DB to Support Dynamic Partitioning and AI-Based Query Prediction

Description:
Our current PostgreSQL-based reporting database for enterprise clients is approaching its vertical scaling limits. The system uses manual sharding per tenant, which is leading to operational bottlenecks and inconsistent query performance across segments.

You are tasked with redesigning the physical and logical schema to support dynamic partitioning based on both tenant activity and time-based access patterns. Investigate the use of table inheritance, BRIN indexes, and partition pruning to optimize performance.

As part of the redesign, integrate query sampling logs into a lightweight ML pipeline (basic LSTM or transformer encoder) that predicts likely query types by hour and pre-warms the relevant partitions into memory. Store prediction results in a sidecar Redis instance.

Migrate historical data (≈3TB) to the new schema without breaking downstream dashboards powered by Apache Superset. Also, coordinate with the DevOps team to fine-tune WAL archiving and replication latency, and with the AI team for pipeline optimization.

Finally, expose a GraphQL endpoint for querying schema metadata, and ensure compatibility with frontend tools that need introspection support.
"""
predict(text, model, tokenizer)

In [ ]:
text = """
Title:
Make gallery

Description:
design a gallery modal of photos with effective and smooth transitions
"""
predict(text, model, tokenizer)

In [ ]:
text = """
Title:
Make gallery

Description:
design a gallery modal of photos with effective and smooth transitions fetching photos from backend and SQL
"""
predict(text, model, tokenizer)

In [ ]:
text = """
Title:
Make classifier

Description:
Fetch dataset from the amazon AWS bucket and make a classifier model for Jira Roles
"""
predict(text, model, tokenizer)

## CONCLUSION
- Roberta performs better for descriptions that do not use keywords specific to the field like SQL, React, Pipeline, etc
- Since SVC is a simple model, it gets confused when descriptions without keywords (like the above teo examples) are used

In [ ]:
trial_df = pd.read_csv("/content/new.csv")
X_trial = trial_df
y_trial = trial_df["role"]
X_trial.drop(columns = ["role"], inplace = True)

In [ ]:
# test_metric(model, X_test, y_test)
predictions, score = test_metric(model, X_trial, y_trial, tokenizer)
# print(predictions)
print("accuracy : ", score)

accuracy :  0.735632183908046


In [ ]:
!zip -r finalmodel.zip /content/summary-training/checkpoint-2800

  adding: content/summary-training/checkpoint-2800/ (stored 0%)
  adding: content/summary-training/checkpoint-2800/trainer_state.json (deflated 78%)
  adding: content/summary-training/checkpoint-2800/optimizer.pt (deflated 77%)
  adding: content/summary-training/checkpoint-2800/rng_state.pth (deflated 25%)
  adding: content/summary-training/checkpoint-2800/model.safetensors (deflated 7%)
  adding: content/summary-training/checkpoint-2800/training_args.bin (deflated 52%)
  adding: content/summary-training/checkpoint-2800/scheduler.pt (deflated 56%)
  adding: content/summary-training/checkpoint-2800/config.json (deflated 52%)
